In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [5]:

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['axes.grid'] = False
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

In [10]:
import pandas as pd
import investpy

### Data Intake

In [6]:
tickers = pd.read_csv("ind_nifty200list.csv")

In [9]:
list(tickers["Symbol"])

['ABB',
 'ACC',
 'AUBANK',
 'ABBOTINDIA',
 'ADANIENT',
 'ADANIGREEN',
 'ADANIPORTS',
 'ADANIPOWER',
 'ATGL',
 'ADANITRANS',
 'AWL',
 'ABCAPITAL',
 'ABFRL',
 'ALKEM',
 'AMBUJACEM',
 'APOLLOHOSP',
 'APOLLOTYRE',
 'ASHOKLEY',
 'ASIANPAINT',
 'ASTRAL',
 'AUROPHARMA',
 'DMART',
 'AXISBANK',
 'BAJAJ-AUTO',
 'BAJFINANCE',
 'BAJAJFINSV',
 'BAJAJHLDNG',
 'BALKRISIND',
 'BANDHANBNK',
 'BANKBARODA',
 'BANKINDIA',
 'BATAINDIA',
 'BERGEPAINT',
 'BEL',
 'BHARATFORG',
 'BHEL',
 'BPCL',
 'BHARTIARTL',
 'BIOCON',
 'BOSCHLTD',
 'BRITANNIA',
 'CGPOWER',
 'CANBK',
 'CHOLAFIN',
 'CIPLA',
 'COALINDIA',
 'COFORGE',
 'COLPAL',
 'CONCOR',
 'COROMANDEL',
 'CROMPTON',
 'CUMMINSIND',
 'DLF',
 'DABUR',
 'DALBHARAT',
 'DEEPAKNTR',
 'DELHIVERY',
 'DEVYANI',
 'DIVISLAB',
 'DIXON',
 'LALPATHLAB',
 'DRREDDY',
 'EICHERMOT',
 'ESCORTS',
 'NYKAA',
 'FEDERALBNK',
 'FORTIS',
 'GAIL',
 'GLAND',
 'GODREJCP',
 'GODREJPROP',
 'GRASIM',
 'FLUOROCHEM',
 'GUJGASLTD',
 'HCLTECH',
 'HDFCAMC',
 'HDFCBANK',
 'HDFCLIFE',
 'HAVELLS',
 '